In [10]:
import mass
import os
import topogenesis as tg
import numpy as np
import pandas as pd
import json
import pyvista as pv

In [11]:
stn_I = tg.create_stencil("von_neumann", 1, 1)
stn_I.set_index([0,0,0], 0)
stn_II = tg.create_stencil("von_neumann", 2, clip=1)
stn_II.set_index([0,0,0], 0)
stencils = {
    "stn_I": stn_I,
    "stn_II": stn_II
    }

In [12]:
data_path = os.path.relpath('data')
prgm_path = os.path.join(data_path, 'program.csv')
program = pd.read_csv(prgm_path, header=[0, 1, 2], index_col=[0], sep=";",converters={i:eval for i in range(1,100)})
program

preferences                              behaviors            
                ent_acc        sun_acc           neighbourhood            
                 desire weight  desire weight         stencils     weights
lobby               1.0    1.0     1.0    0.0          [stn_I]         [1]
roof_garden         1.0    0.0     1.0    1.0  [stn_I, stn_II]  [0.5, 0.5]
workshop            1.0    0.5     1.0    0.5          [stn_I]         [1]

In [13]:
avail_lattice = tg.lattice_from_csv(os.path.join(data_path, 'voxelized_envelope.csv'))

env_lattices = {l_name: tg.lattice_from_csv(os.path.join(data_path,l_name+'.csv')) for l_name in set(program["preferences"].columns.get_level_values(0))}

In [14]:
agents_dict = mass.utilities.df_to_nested_dict(program)
for aid, a in enumerate(agents_dict):
    agents_dict[a]["aid"] = aid

In [16]:
agents_dict

{'lobby': {'preferences': {'ent_acc': {'desire': 1.0, 'weight': 1.0},
   'sun_acc': {'desire': 1.0, 'weight': 0.0}},
  'behaviors': {'neighbourhood': {'stencils': ['stn_I'], 'weights': [1]}},
  'aid': 0},
 'roof_garden': {'preferences': {'ent_acc': {'desire': 1.0, 'weight': 0.0},
   'sun_acc': {'desire': 1.0, 'weight': 1.0}},
  'behaviors': {'neighbourhood': {'stencils': ['stn_I', 'stn_II'],
    'weights': [0.5, 0.5]}},
  'aid': 1},
 'workshop': {'preferences': {'ent_acc': {'desire': 1.0, 'weight': 0.5},
   'sun_acc': {'desire': 1.0, 'weight': 0.5}},
  'behaviors': {'neighbourhood': {'stencils': ['stn_I'], 'weights': [1]}},
  'aid': 2}}

In [15]:
env = mass.environment(avail_lattice, env_lattices, agents_dict, stencils)

Valid environment created, shape =(16, 15, 5)


In [ ]:
occ_frames = [np.copy(env.occ_lattice)]

max_iteration = 10
# Simulation Loop
for i in range(max_iteration):
    # Agent Loop
    for a_name, a in env.agents.items():

        # Neighbourhood Evaluation
        ##########
        neigh_lat = avail_lattice * 0.0
        for s, w in zip(
            a.behaviors["neighbourhood"]["stencils"], 
            a.behaviors["neighbourhood"]["weights"]):
            # find and aggregate neighs
            neigh_lat += a.bhv_find_neighbour(s, env) * w    

        # extract neigh values for agent
        neigh_eval_lat = neigh_lat * a.eval_lat

        # Occupation
        ##########
        a.bhv_occupy(env, np.unravel_index(neigh_eval_lat.argmax(), neigh_eval_lat.shape))
    #############
    # End of agent loop
    occ_frames.append(np.copy(env.occ_lattice))
#############
# End of simulation loop


In [ ]:
p = pv.Plotter(notebook=True)

base_lattice = env.avail_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# # adding the avilability lattice
# init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = occ_frames[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([-0.1, len(env.agents) - 0.9])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

    return

p.add_slider_widget(create_mesh, [0, len(occ_frames)], title='Time', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)